In [7]:
from config import setup
setup()

In [25]:
import math
import json
import networkx as nx
from docplex.cp.model import CpoModel
from docplex.cp.model import *

# Comme on ne peut pas définir la longueur d'un arc à partir des fichiers générés ...
VALEUR_ARBITRAIRE_ARC_SORTIE = 1

In [11]:
def read_evac(filename):
    
    with open(filename,"r") as file:
        content = file.readlines()
    
    if len(content) > 0:
        
        nbzones, nbarcs = content[0].split(" ")
        content.remove(content[0])
    
        lineszones = content[0:int(nbzones)]
        linesarcs  = content[int(nbzones):int(nbzones)+int(nbarcs)]
        
        E = []
        
        for line in lineszones:
            personnes, paquets, datemax = line.split(" ")
            E.append([int(personnes),int(paquets),int(datemax)])
            
        A = []
        
        for line in linesarcs:
            
            content = line.split(" ")
            capacite = content[0]
            content = content[2:]
            
            groups = []
            
            for i in range(0,len(content),2):
                numgroupe = content[i]
                datemax = content[i+1]
                
                groups.append([int(numgroupe),int(datemax)])
                
            A.append([int(capacite),groups])
            
        return E,A
        
        
# import os
# os.listdir()

E,A = read_evac("evacsim-master/data/test_10_25_2_10.evac")

print(*E, sep = "\n")
print(*A, sep = "\n")

[4455, 70, 68]
[509, 70, 22]
[4584, 71, 128]
[2902, 71, 122]
[2808, 71, 84]
[2744, 70, 22]
[3982, 3982, 50]
[1471, 70, 122]
[1783, 71, 22]
[358, 70, 68]
[71, [[1, 0], [8, 0]]]
[224, [[2, 17], [4, 35], [5, 30]]]
[70, [[1, 3], [6, 3], [8, 3]]]
[132, [[4, 29], [5, 24]]]
[132, [[0, 0], [3, 8], [9, 0]]]
[249, [[0, 18], [1, 23], [2, 30], [3, 26], [4, 48], [5, 43], [6, 23], [8, 23], [9, 18]]]
[132, [[3, 8], [9, 0]]]
[146, [[0, 1], [1, 6], [3, 9], [6, 6], [8, 6], [9, 1]]]


In [28]:
def graph2list(filename):
    
    E,A = read_evac("evacsim-master/data/test_10_25_2_10.evac")
    
    ########################## TRAITEMENT DES SOMMETS ##########################
    
    new_E = [] # Ensemble de sommets à évacuer
    
    for i_k in range(len(E)):
        w_k, h_k, d_k = E[i_k]
        p_k = math.ceil(w_k / h_k)
        A_k = []
        s_k = None
        
        k = [
            i_k,  # Identifiant du sommet
            w_k,  # Nombre de personnes à évacuer
            h_k,  # Taux d'évacuation d'un sommet
            d_k,  # Date à laquelle le sommet crame
            p_k,  # Durée d'évacuation d'un sommet
            A_k,  # Liste des sommets pour le chemin d'évacuation
            s_k,  # Date début évacuation d'un sommet
        ]
        
        new_E.append(k)
        
    ########################## TRAITEMENT DES ARCS ##########################
    
    new_A = []
    
    for i_e in range (len(A)):
        c_e, groups = A[i_e]
        in_e = None
        out_e = None
        l_e = None

        for i_k, b_e in groups:
            
            e = [    # Arc d'un chemin d'évacuation d'un sommet
                i_e, # Identifiant de l'arc
                b_e, # Date min de passage
            ]
            
            k = new_E[i_k]
            A_k = k[5]
            
            if len(A_k) == 0:
                A_k.append(e)
            else:
                added = False
                for i in range (len(A_k)):
                    if A_k[i][1] > e[1]:
                        A_k.insert(i,e)
                        added = True
                if added == False:
                    A_k.append(e)
            
            k[5] = A_k
            new_E[i_k] = k

        e = [
            i_e,     # Identifiant de l'arc
            c_e,     # Capacité de l'arc en personnes par unité de temps
            in_e,    # Identifiant du sommet entrant
            out_e,   # Identifiant du sommet sortant
            l_e,     # Longueur de l'arc
        ]
        
        new_A.append(e)
        
    ####################### IDENIFICATION DES SOMMETS #######################
    
    # Cette partie va servir à trouver et compléter la liste des sommets
    # intermédiaires et de sortie, tout en complétant les in/out des arcs
        
    new_T = [] # Ensemble de sommets de transfert
    new_S = [] # Ensemble de sommets séurisés
    
    k_sommets = len(new_E)
    k_arcs = []
    
    for k in new_E:
        A_k = k[5]
        parcours = k[0]
        for i in range (len(A_k)):
            e = new_A[A_k[i][0]]
            
            if e[0] in k_arcs:
                
                #Arc connu donc rien à faire
                continue
            
            else:
                
                #On ajoute le sommet d'entrée de l'arc
                e[2] = parcours
                
                #On ajoute l'arc aux arc parcourus
                k_arcs.append(e[0])
                
                #Contrôle de l'arc suivant
                if i+1 < len(A_k):

                    #Il y a bien un arc suivant, on a affaire à un sommet intermédiaire
                    e_suiv = new_A[A_k[i+1][0]]
                    
                    #On peut donc connaître la longueur de l'arc
                    e[4] = A_k[i+1][1] - A_k[i][1]

                    if e_suiv[0] in k_arcs:

                        #Arc connu donc on connait son entrée, donc celui de sortie de l'arc actuel
                        e[3] = e_suiv[2]

                    else:

                        #Arc inconnu donc on a découvert un nouveau sommet intermédiaire
                        e[3] = k_sommets
                        new_T.append(k_sommets)
                        k_sommets += 1

                    #On prépare le sommet de départ pour le prochain arc
                    parcours = e[3]

                else:

                    #Pas de sommet suivant donc on a atteint un sommet de sécurité
                    e[3] = k_sommets
                    new_S.append(k_sommets)
                    k_sommets += 1
                    
                    #On ne peut donc pas connaître la longueur de l'arc, on la met arbitraitement à 1
                    e[4] = VALEUR_ARBITRAIRE_ARC_SORTIE
                        
                #Mise à jour de e
                new_A[e[0]] = e
                
    # TODO Bug sur le sommet 3 et arc 6 (qui a la même date au plus tôt que l'arc 4)
    # On fait le choix de l'ignorer pour le moment, on a quand même un graphe utilisable
            
    ########################## TRAITEMENT DU GRAPHE ##########################
    
    X = [       # Ensemble des sommets du graphe
        new_E,  # Ensemble de sommets à évacuer
        new_T,  # Ensemble de sommets de transfert
        new_S,  # Ensemble de sommets séurisés
    ]
    
    G = [       # Notre graphe
        X,      # Ensemble des sommets du graphe
        new_A,  # Ensemble des arêtes du graphe
    ]
    
    return G

G_list = graph2list("projet/evacsim-master/data/test_10_25_2_10.evac")

print(json.dumps(G_list, indent=1))


[
 [
  [
   [
    0,
    4455,
    70,
    68,
    64,
    [
     [
      4,
      0
     ],
     [
      7,
      1
     ],
     [
      5,
      18
     ]
    ],
    null
   ],
   [
    1,
    509,
    70,
    22,
    8,
    [
     [
      0,
      0
     ],
     [
      2,
      3
     ],
     [
      7,
      6
     ],
     [
      5,
      23
     ]
    ],
    null
   ],
   [
    2,
    4584,
    71,
    128,
    65,
    [
     [
      1,
      17
     ],
     [
      5,
      30
     ]
    ],
    null
   ],
   [
    3,
    2902,
    71,
    122,
    41,
    [
     [
      4,
      8
     ],
     [
      6,
      8
     ],
     [
      7,
      9
     ],
     [
      5,
      26
     ]
    ],
    null
   ],
   [
    4,
    2808,
    71,
    84,
    40,
    [
     [
      3,
      29
     ],
     [
      1,
      35
     ],
     [
      5,
      48
     ]
    ],
    null
   ],
   [
    5,
    2744,
    70,
    22,
    40,
    [
     [
      3,
      24
     ],
     [
      1,
     

In [29]:
def list2geosafe(G_list):
    
    nodes_state = []
    edges_state = []
    
    for eps in G_list[0][0]:
        nodes_state.append((eps[0], {'eps': True,  'r': False, 'W': eps[1], 'd': eps[3]}))
        
    for trans in G_list[0][1]:
        nodes_state.append((trans, {'eps': False,  'r': False, 'W': 0, 'd': 0}))
        
    for r in G_list[0][2]:
        nodes_state.append((r, {'eps': False,  'r': True, 'W': 0, 'd': 0}))
        
    for edge in G_list[1]:
        edges_state.append((edge[2], edge[3], {'l' : edge[4]}))
    
    G = nx.DiGraph()
    G.add_nodes_from(nodes_state)
    G.add_edges_from(edges_state)
    
    return G
        
G = list2geosafe(G_list)
    